In [3]:
import numpy as np

In [4]:
class Expression:
    def __init__(self, ele_func, sub_expr1, sub_expr2=None):
        self._ele_func  = ele_func
        self._sub_expr1 = sub_expr1
        self._sub_expr2 = sub_expr2
    
    def evaluation_at(self, val_dict):
        
        # self._sub_expr2 is None implies that self._ele_func is an unary operator
        if self._sub_expr2 is None: 
            return self._ele_func.evaluation_at(
                self._sub_expr1, val_dict)
        
        # self._sub_expr2 not None implies that self._ele_func is a binary operator
        else:
            return self._ele_func.evaluation_at(
                self._sub_expr1, self._sub_expr2, val_dict)
    
    def derivative_at(self, var, val_dict):
        
        # sub_expr2 is None implies that _ele_func is an unary operator
        if self._sub_expr2 is None:
            return self._ele_func.derivative_at(
                self._sub_expr1, var, val_dict)
        
        # sub_expr2 not None implies that _ele_func is a binary operator
        else:
            return self._ele_func.derivative_at(
                self._sub_expr1, self._sub_expr2, var, val_dict)
    
    def __add__(self, another):
        if isinstance(another, Expression):
            return Expression(Add, self, another)
        # if the other operand is not an Expression, then it must be a number
        # the number then should be converted to a Constant
        else:
            return Expression(Add, self, Constant(another))
    
    def __radd__(self, another):
        if isinstance(another, Expression):
            return Expression(Add, another, self)
        else:
            return Expression(Add, Constant(another), self)
    
    def __sub__(self, another):
        if isinstance(another, Expression):
            return Expression(Sub, self, another)
        else:
            return Expression(Sub, self, Constant(another))
    
    def __rsub__(self, another):
        if isinstance(another, Expression):
            return Expression(Sub, another, self)
        else:
            return Expression(Sub, Constant(another), self)
        

    def __mul__(self, another):
        if isinstance(another, Expression):
            return Expression(Mul,self,another)
        else:
            return Expression(Mul, self, Constant(another))

    def __rmul__(self, another):
        if isinstance(another, Expression):
            return Expression(Mul,another,self)
        else:
            return Expression(Mul, Constant(another),self)
    
    def __truediv__(self, another):
        if isinstance(another, Expression):
            return Expression(Div,self,another)
        else:
            return Expression(Div, self, Constant(another))

    def __rtruediv__(self, another):
        if isinstance(another, Expression):
            return Expression(Div,another,self)
        else:
            return Expression(Div, Constant(another),self)
    
    def __pow__(self,another):
        if isinstance(another, Expression):
            return Expression(Pow,self,another)
        else:
            return Expression(Pow, self, Constant(another))
    
    def __rpow__(self,another):
        if isinstance(another, Expression):
            return Expression(Pow,another,self)
        else:
            return Expression(Pow, Constant(another),self)

In [5]:
class Variable(Expression):
    def __init__(self):
        return
    
    def evaluation_at(self, val_dict):
        return val_dict[self]
    
    def derivative_at(self, var, val_dict):
        return 1.0 if var is self else 0.0

In [6]:
class Constant(Expression):
    def __init__(self, val):
        self.val = val
        
    def evaluation_at(self, val_dict):
        return self.val
    
    def derivative_at(self, var, val_dict):
        return 0.0

In [7]:
class Add:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) + \
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) + \
               sub_expr2.derivative_at(var, val_dict)

In [8]:
class Sub:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) - \
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) - \
               sub_expr2.derivative_at(var, val_dict)

In [9]:
class Mul:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) *\
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) * \
               sub_expr2.evaluation_at(val_dict)+ \
               sub_expr1.evaluation_at(val_dict) *\
               sub_expr2.derivative_at(var, val_dict)

In [10]:
class Div:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) /\
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return  sub_expr1.derivative_at(var, val_dict) / \
                sub_expr2.evaluation_at(val_dict)+ \
                sub_expr1.evaluation_at(val_dict) *\
                sub_expr2.derivative_at(var, val_dict)/\
                sub_expr2.evaluation_at(val_dict)/\
                sub_expr2.evaluation_at(val_dict)

In [45]:
class Pow:
    
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) **\
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    #f(x)^g(x) * g‘(x)  * ln( f(x) )+ f(x)^( g(x)-1 ) * g(x) * f’(x) 
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return  sub_expr1.evaluation_at(val_dict)** \
                sub_expr2.evaluation_at(val_dict)* \
                sub_expr2.derivative_at(var, val_dict)*\
                np.log(sub_expr1.evaluation_at(val_dict))+ \
                sub_expr1.evaluation_at(val_dict) **\
                (sub_expr2.evaluation_at(val_dict)-1)*\
                sub_expr2.evaluation_at(val_dict)*\
                sub_expr1.derivative_at(var, val_dict)

In [46]:
class Exp:
    @staticmethod
    def evaluation_at(sub_expr1, val_dict):
        return np.exp(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) * np.exp(sub_expr1.evaluation_at(val_dict))

In [47]:
def exp(expr):
    return Expression(Exp, expr)

Let $f(a, b, c) = e^{a-b+c}$, at $(a, b, c) = (1.0, 2.0, 3.0)$, that should be $e^{2} \approx 7.389$.

In [48]:
a = Variable()
b = Variable()
c = Variable()
f = exp(a-b+c)

In [49]:
f.evaluation_at({a: 1.0, b: 2.0, c: 3.0})

7.3890560989306504

$\dfrac{\partial f}{\partial b} = -e^{a-b+c}$, at $(a, b, c) = (1.0, 2.0, 3.0)$, that should be $-e^{2} \approx -7.389$.

In [50]:
f.derivative_at(b, {a: 1.0, b: 2.0, c: 3.0})

-7.3890560989306504

$\dfrac{\partial f}{\partial a} = e^{a-b+c}$, at $(a, b) = (1.0, 2.0, 3.0)$, that should be $e^{2} \approx 7.389$.

In [51]:
f.derivative_at(a, {a: 1.0, b: 2.0, c: 3.0})

7.3890560989306504

Let $g(x, y) = x + e^{y-1}$, at $(x, y) = (1.0, 2.0)$, that should be $1+e \approx 3.718$.

In [52]:
x = Variable()
y = Variable()
g = x + exp(y-1)

In [53]:
g.evaluation_at({x: 1.0, y: 2.0})

3.7182818284590451

$\dfrac{\partial g}{\partial x} = 1$

In [54]:
g.derivative_at(x, {x: 1.0, y: 2.0})

1.0

$\dfrac{\partial g}{\partial y} = e^{y-1}$, at $(x, y) = (1.0, 2.0)$, that should be $e^{1} \approx 2.718$.

In [55]:
g.derivative_at(y, {x: 1.0, y: 2.0})

2.7182818284590451

#### Test for Multiple

In [56]:
x = Variable()
y = Variable()
g = x*y
f = x/y

In [57]:
g.evaluation_at({x: 2.0, y: 2.0})

4.0

In [58]:
g.derivative_at(x, {x: 1.0, y: 2.0})

2.0

In [59]:
f.evaluation_at({x: 3.0, y: 2.0})

1.5

In [60]:
f.derivative_at(y, {x: 4.0, y: 2.0})

1.0

In [61]:
a = 3.4

In [62]:
2/3

0.6666666666666666

### Test for Power

In [63]:
x = Variable()
y = Variable()
g = x**y


In [66]:
g.evaluation_at({x: 3.0, y: 2.0})

9.0

In [71]:
g.derivative_at(x, {x: 3.0, y: 2.0})

6.0

In [82]:
%%bash
cd /Users/ting/Dropbox\ \(MIT\)/Fall\ 2018/CS207/cs207-FinalProject/


/Users/ting/Dropbox (MIT)/Fall 2018/CS207/cs207-FinalProject


'/Users/ting/Dropbox (MIT)/Fall 2018/CS207/cs207-FinalProject/code_explore'

In [79]:

import pytest
import numpy as np
import autodiff.forward as fwd


def test_multiple_constant():
    x = fwd.Variable()
    assert (2.0*x).derivative_at(x,{x:3.0}) == 2.0
    assert (x*2.0).derivative_at(x,{x:3.0}) == 2.0

def test_divide_constant():
    x = fwd.Variable()
    assert (x/2.0).derivative_at(x,{x:3.0}) == 0.5
    assert (2.0/x).derivative_at(x,{x:3.0}) == -2/9.0

def test_multiple():
    x = fwd.Variable()
    y = fwd.Variable()
    f = x*y
    assert f.evaluation_at({x: 3.0, y: 2.0}) == 6.0
    assert f.derivative_at(x, {x: 3.0, y: 2.0}) == 2.0
    assert f.derivative_at(y, {x: 3.0, y: 2.0}) == 3.0

def test_divide():
    x = fwd.Variable()
    y = fwd.Variable()
    f = x/y
    assert f.evaluation_at({x: 3.0, y: 2.0}) == 1.5
    assert f.derivative_at(x, {x: 3.0, y: 2.0}) == 1/2.0
    assert f.derivative_at(y, {x: 3.0, y: 2.0}) == -0.75

def test_power():
    x = fwd.Variable()
    y = fwd.Variable()
    f = x**y
    assert f.evaluation_at({x: 3.0, y: 2.0}) == 9.0
    assert f.derivative_at(x, {x: 3.0, y: 2.0}) == 6.0
    assert f.derivative_at(y, {x: 3.0, y: 2.0}) == np.log(3.)*3**2


bash: line 2: import: command not found
bash: line 3: import: command not found
bash: line 4: import: command not found
bash: line 7: syntax error near unexpected token `('
bash: line 7: `def test_multiple_constant():'


In [74]:
test_power()

NameError: name 'fwd' is not defined